>
> # MaaSSim tutorial

 ### parallel simulations

In [102]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
from dotmap import DotMap
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

prepare

In [104]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate, simulate_parallel
from MaaSSim.traveller import travellerEvent
from MaaSSim.driver import driverEvent

In [105]:
params = get_config('../../data/config/delft.json')  # load configuration

In [106]:
params.paths.dumps = 'dumps'
params.simTime = 4
params.parallel.nThread = 4
params.parallel.nReplications = 2

In [107]:
inData = load_G(inData, params)  # load network graph 

ParserError: Error tokenizing data. C error: Expected 1 fields in line 22, saw 2


In [78]:
space = DotMap()
space.nP = list(range(50,501,50))
space.nV = list(range(20,201,20))

simulate

In [79]:
simulate_parallel(params = params, search_space = space)

07-10-20 14:52:20-WARNING-Setting up 4h simulation at 2020-10-07 12:52:40 for 20 vehicles and 50 passengers in Delft, Netherlands
07-10-20 14:52:21-WARNING-Setting up 4h simulation at 2020-10-07 12:52:40 for 140 vehicles and 50 passengers in Delft, Netherlands
07-10-20 14:52:21-WARNING-Setting up 4h simulation at 2020-10-07 12:52:40 for 80 vehicles and 100 passengers in Delft, Netherlands
07-10-20 14:52:21-WARNING-Setting up 4h simulation at 2020-10-07 12:52:40 for 200 vehicles and 100 passengers in Delft, Netherlands
07-10-20 14:52:21-WARNING-simulation time 0.9 s
07-10-20 14:52:22-WARNING-simulation time 0.8 s
07-10-20 14:52:22-WARNING-assertion tests for simulation results - passed
07-10-20 14:52:22-WARNING-assertion tests for simulation results - passed
07-10-20 14:52:22-WARNING-simulation time 0.9 s
-nP_50-nV_20-dt_20201007145220694768 2020-10-07 14:52:22.282243 end
-nP_50-nV_140-dt_20201007145220835297 2020-10-07 14:52:22.354110 end
07-10-20 14:52:22-WARNING-assertion tests for s

### collect

demand structure

In [80]:
from MaaSSim.utils import collect_results  

In [81]:
res = collect_results(params.paths.dumps)

In [101]:
res.trips.t.mean(), res.trips.t.std()

(6181.140875, 4593.540236993354)

In [ ]:

raw = sim.runs[0].trips[sim.runs[0].trips.pax == trip_id]  # this is his record within simulation

In [98]:
wait_times = list()
trips = res.trips
for repl in trips.dt.unique():
    trips_repl = trips[trips.dt == repl]
    for pax in trips_repl.pax.unique():
        raw = trips_repl[trips_repl.pax == pax]
        wt = pd.to_timedelta(raw[raw.event == travellerEvent.DEPARTS_FROM_PICKUP.name].squeeze().t -  raw[raw.event == travellerEvent.REQUESTS_RIDE.name].squeeze().t,'s')
        wait_times.append({'pax':pax,
                           'nP':raw.nP.max(),
                           'nV':raw.nV.max(),
                          'wt':wt})
wait_times = pd.DataFrame(wait_times)

In [87]:
res.trips

In [85]:
res.pax_exp

,pax,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,RECEIVES_OFFER,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,nP,nV,dt
0,0,15.0,447.0,20.0,30.0,244.0,NaN,10.0,0,0,0,0,0,0,0,300,80,20201007145312020164
1,1,15.0,36.0,20.0,30.0,41.0,121.0,10.0,0,0,0,0,0,0,0,300,80,20201007145312020164
2,2,15.0,462.0,20.0,30.0,311.0,646.0,10.0,0,0,0,0,0,0,0,300,80,20201007145312020164
3,3,15.0,37.0,20.0,30.0,60.0,1876.0,10.0,0,0,0,0,0,0,0,300,80,20201007145312020164
4,4,15.0,445.0,20.0,30.0,62.0,2127.0,10.0,0,0,0,0,0,0,0,300,80,20201007145312020164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,15,15.0,382.0,20.0,30.0,17.0,10233.0,10.0,0,0,0,0,0,0,0,400,120,20201007145319948470
16,16,15.0,279.0,20.0,30.0,193.0,11922.0,10.0,0,0,0,0,0,0,0,400,120,20201007145319948470
17,17,15.0,209.0,20.0,30.0,19.0,12918.0,10.0,0,0,0,0,0,0,0,400,120,20201007145319948470
18,18,15.0,131.0,20.0,30.0,55.0,13031.0,10.0,0,0,0,0,0,0,0,400,120,20201007145319948470


-------------
Rafal Kucharski, 2020, Delft, r.m.kucharski@tudelft.nl